In [1]:
import numpy as np
import pandas as pd

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# SECOND PHASE parsing
# translate the outlook into output requirements
file_path = "data/outlook_vehicles_stock.csv"
outlook = pd.read_csv(file_path)
outlook = pd.melt(outlook, id_vars=['category','region','year'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicle', value_name='outlook')

In [3]:
file_path = "inputs/input_welltotank.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['Product'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicle', value_name='requirement')
requirement.rename(columns={'Product': 'sector'},inplace=True)

In [4]:
outlook_requirement = pd.merge(outlook, requirement, on='vehicle', how='outer')
outlook_requirement["value"] = outlook_requirement["outlook"] * outlook_requirement["requirement"]

In [5]:
outlook_sectors = outlook_requirement.pivot_table(index=['category', 'region', 'year', 'sector'],
                          columns='vehicle',
                          values='value',
                          aggfunc='sum')

In [6]:
scenario_list = ["Projection-APS","Projection-STEPS"]
#scenario = scenario_list[0]
for scenario in scenario_list:
    outlook_scenario = outlook_sectors.reset_index()
    outlook_scenario = outlook_scenario[(outlook_scenario.category==scenario)&(outlook_scenario.region=="Europe")]
    outlook_scenario["region"] = "EU"
    outlook_scenario = outlook_scenario[["year","region","sector","ICEV","PHEV","BEV","FCEV"]]

    file_name = "data/outlook_welltotank_"+scenario+".csv"
    outlook_scenario.to_csv(file_name)